In [18]:
# set of (frozen) sets
sos = lambda *S: set(frozenset(A) for A in S)

# Probability Space

**<u>Definition</u>**

A probability space consists of three objects $(\Omega, \mathcal{F}, P)$:

* $\Omega$ &ndash; sample space: a nonempty set

* $\mathcal{F}$ &ndash; $\sigma$-algebra on $\Omega$: a set of sets

* $P$ &ndash; probability measure on $\mathcal{F}, \Omega$: a function mapping $P: \mathcal{F} \to [0, 1]$

## Sample Space $\Omega$

In [2]:
Ω = {1, 2, 3}

## Algebra $\mathcal{F}_{a}$

**<u>Definition</u>**

1. Sample space: $\Omega \in \mathcal{F}_{a}$ (and also empty set $\emptyset \in \mathcal{F}_a$).

2. Closed under complements: $A \in \Omega \implies A^c \in \mathcal{F}_{a}$.

3. Closed under finite union: $A, B \in \Omega \implies A \cup B \in \mathcal{F}_{a}$.

## $\sigma$-Algebra $\mathcal{F}$

**<u>Definition</u>**

Similar definition as that of an algebra, but closed under countable (infinite) union.

1. Sample space: $\Omega \in \mathcal{F}$ (and also empty set $\emptyset \in \mathcal{F}$).


2. Closed under complements: $A \in \Omega \implies A^c \in \mathcal{F}$.

3. Closed under countable (infinite) union: $\bigcup\limits_{i=1}^{\infty}A_i \in F, A_i \in \Omega.$

In [27]:
def algebra(generating_set, sample_space):

    F = [S for S in generating_set if S.issubset(sample_space)]
    
    if sample_space not in F:
        F.append(sample_space)

    if set() not in F:
        F.append(set())

    while True:
        
        F_old = F.copy()

        for A in F:
            # closure under complements
            A_comp = sample_space - A
            if A_comp not in F:
                F.append(A_comp)

            for B in F:
                # closure under finite union
                union = A.union(B)
                if union not in F:
                    F.append(union)

        if sos(*F_old)==sos(*F):
            return sos(*F)

In [30]:
algebra(sos({1}), Ω)

{frozenset(), frozenset({1}), frozenset({2, 3}), frozenset({1, 2, 3})}

## The Power Set

Given a set $S$, its power set $2^S$  is the set of all subsets of $S$. The power set of a sample space $\Omega$ is the largest $\sigma$-algebra on $\Omega$.

In [5]:
def power_set(S):
    
    ps = [{x} for x in S]
    
    for x in ps:
        for y in ps:
            union = x.union(y)
            if union not in ps:
                ps.append(union)
    
    # add empty set
    ps.append(set())
    
    return sos(ps)

In [6]:
ps = power_set(Ω)
len(ps)

In [7]:
x = [0]

for y in x:
    x.append(y+1)
    if len(x) > 100:
        break
        
print(x)

# Probability Measure $P$

In [39]:
def P(Fset, Ω):
    return len(Fset) / len(Ω)

0.6666666666666666

# Borel Set

In [8]:
class Naturals(object):
    
    def __init__(self, start=0):
        self.start = start
        self._gen = self.generator(start)
        
    def __repr__(self):
        return 'ℕ'
        
    def __contains__(self, other):
            
        if int(other) == other:
            return other >= self.start
        else:
            return False
    
    def generator(self, n):
        yield n
        yield from self.generator(n+1)
        
    def next_n(self):
        return next(self._gen)
        
N = Naturals()
1 in N

True

# Random Variable


A random variable is a function $X : \Omega \to \mathbb{R}$ such that, given probability space $(\Omega, \mathcal{F}, P)$:

$$
\lbrace \omega \in \Omega : X(\omega) \le x \rbrace \in \mathcal{F} \quad \forall x \in \mathbb{R}.
$$

in other words, for all $x\in \mathbb{R}$, the inverse image of $X$ on interval $(-\infty, x]$ is in $F$:

$$
\lbrace X \leq x \rbrace = X^{-1}((-\infty, x]) \in \mathcal{F}.
$$

In [9]:
import time

In [10]:
def X(ω):
    
    return 2* ω

def X_inv(X, x, Ω):
    
    res = set()
    for ω in Ω:
        if X(ω) <= x:
            res.add(ω)
    return res


def is_rv(X, Ω, F, t=1.0, return_xlim=False):
    '''
    Returns boolean if X is a r.v.
    
        Parameters:
            X (function):     A random variable candidate
            Ω (set):          The sample space
            F (set):          Sigma algebra on Ω (set of set)
            t (int or float): Time limit for testing (default is 1.0)
            
        Returns:
            is_rv (bool): Whether X is a r.v.
    '''
    
    t0, x= time.time(), 0
    while time.time()-t0 < 1:
        
        if X_inv(X, x, Ω) not in F or X_inv(X, -x, Ω) not in F:
            return False
        
        x+=1
    
    if return_xlim:
        return True, [-x, x]
    else:
        return True
    

In [11]:
is_rv(X, Ω, ps)

True

In [12]:
class Infinity(object):
    '''
    Infinity ✨
    '''
    def __init__(self, cardinality=1):
        
        self.card = cardinality
        self.pos = True
        
    def __repr__(self):
        if self.pos:
            return f'∞ (cardinality {self.card})'
        else:
            return f'-∞ (cardinality {self.card})'
    
    def copy(self):
        '''
        Return copy of self
        '''
        obj = type(self).__new__(self.__class__) # create a new instance
        obj.__dict__.update(self.__dict__) # update dictionary
        return obj
    
    def __add__(self, other):
        if type(self)==type(other):
            if other.card > self.card:
                return other
            else:
                return self
        else:
            return self
        
    def __radd__(self, other):
        return self + other
    
    def __neg__(self):
        res = self.copy()
        res.pos = False
        return res
    
    def __sub__(self, other):
        if type(self)==type(other):
            if other.card > self.card:
                return -other
            else:
                return self
        else:
            return self
        
    def __rsub__(self, other):
        return -(self-other)
        
    def __mul__(self, other):
        if type(self)==type(other):
            if other.card > self.card:
                return other
            else:
                return self
        else:
            return self
        
    def __rmul__(self, other):
        return self * other
    
    def __lt__(self, other):
        
        if type(self)==type(other):
            return self.card < other.card
        else:
            return False
        
    def __le__(self, other):
        if type(self)==type(other):
            return self.card <= other.card
        else:
            return False
    
    def __gt__(self, other):
        if type(self)==type(other):
            return self.card > other.card
        else:
            return True
        
    def __ge__(self, other):
        if type(self)==type(other):
            return self.card >= other.card
        else:
            return True

infty = Infinity()